In [1]:
from bs4 import BeautifulSoup
import requests
import re

#Declare tuples for variable part of url
forums_page_tup=[['anxiety',1],['depression',2],['ptsd-and-trauma',3],['suicidal-thoughts-and-self-harm',4]]

#fixed parts of url
sub_url='/bd-p/c1-sc2-b'
base_url='https://forums.beyondblue.org.au/t5/'
page_string='/page/'

#empty dictionary for kind of category of mental health problem
all_urls = {}

#Categories of mental problem
cat_urls = ['Anxiety', 'Depression', 'PTSD and trauma', 'Suicidal thoughts and self-harm']

In [2]:
#Define function to extract all url addreses to scrape posts, return dictionary with kind of mental problem, 
#base url, number of pages and all url addresses for given problem

def urls_generator(basic, sub, category, category_list):
    
    all_urls = {}
    

    for i in range(len(cat_urls)):
        num_cat_urls=[]

        cat=str(category[i][0])
        num=str(category[i][1])
        #Create base url first page of forum
        base_cat_url=basic+cat+sub+num
        num_cat_urls.append(base_cat_url)


        page_content=requests.get(base_cat_url).text
        soup = BeautifulSoup(page_content, "html.parser")
        #Extract last page to define how many pages scrape
        last_page=soup.find('li',{'class':re.compile('lia-paging-page-last')})
        if last_page!=None:
            last_page=int(soup.find('li',{'class':re.compile('lia-paging-page-last')}).text)
        else:
            continue
        urls_pages=list(range(2,last_page+1))
        all_urls[cat_urls[i]] = {}
        all_urls[cat_urls[i]]['url'] = num_cat_urls
        all_urls[cat_urls[i]]['pages'] = urls_pages
    for c in cat_urls:
        
        page_list= all_urls[c]['pages']

        pages_to_url=[all_urls[c]['url'][0]]
        for p in page_list:
            basic_page=all_urls[c]['url'][0]

            page_url=basic_page+page_string+str(p)



            pages_to_url.append(page_url)
        all_urls[c]['page_list']=pages_to_url
    return all_urls



In [3]:
urls=urls_generator(base_url,sub_url,forums_page_tup,cat_urls)

In [4]:
#Now we can scrape posts with title and category

from tqdm import tqdm
posts=[]
for i in range(len(cat_urls)):
  #Define progress bar and description
  pbar=tqdm(urls[cat_urls[i]]['page_list'])
  pbar.set_description("Progress for "+str(cat_urls[i])+" posts "+str(i+1)+"/"+str(len(cat_urls)))
  dates=[]
  titles=[]
  post_texts=[]
  for web in pbar:
    
    page_content=requests.get(web).text
    soup = BeautifulSoup(page_content, "html.parser")
    
    
    
    #find title of post
    for t in soup.find_all("a", href=re.compile("/td-p/")):
      titles.append(t.get('title'))
    #find post text
    for p in soup.find_all('p', {'body-text'}):
      text=p.text
      post_texts.append(re.sub('\n',' ',text).strip())
    #find date of post
    for d in soup.find_all('time'):
      dates.append(d.text)
      
    
  posts.append([list(x) for x in zip([cat_urls[i]]*len(titles),dates,titles,post_texts)])
  print("Scraped "+str(len(post_texts)))
    

    
    


    

Progress for Anxiety posts 1/4: 100%|█████████| 639/639 [12:34<00:00,  1.18s/it]


Scraped 7662


Progress for Depression posts 2/4: 100%|██████| 551/551 [11:03<00:00,  1.20s/it]


Scraped 7158


Progress for PTSD and trauma posts 3/4: 100%|█| 169/169 [03:07<00:00,  1.11s/it]


Scraped 1858


Progress for Suicidal thoughts and self-harm posts 4/4: 100%|█| 83/83 [01:23<00:

Scraped 990


In [14]:
from itertools import chain
posts_new=list(chain(*posts))
len(posts_new)

17668

In [15]:
import pandas as pd

mental_health_df=pd.DataFrame(posts_new, columns=['category','date','title','text'])

In [18]:
#In date column of df some dates are in text format like yesterday, week ago or Monday, Tuesday etc.

unique_dates=[]
for date in mental_health_df['date'].tolist():
  re_date=re.search('^[0-9]{1,2}-[0-9]{1,2}-[0-9]{4}$',date)
  if re_date==None:
    d_ago=re.sub('[0-9]','',date).strip()
    if d_ago in unique_dates:
      continue
    unique_dates.append(d_ago)


In [19]:
unique_dates

['yesterday',
 'weeks ago',
 'Tuesday',
 'Sunday',
 'Friday',
 'a week ago',
 'a month ago',
 'Monday',
 'hours ago',
 'Saturday',
 'Thursday']

In [20]:
from datetime import datetime, timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import re
weekdays={'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6}
#For dates expressed as day of week, function to parse day to date in format dd-mm-YYYY
def parse_day_to_date(day):

      date_weekday=None
      today=date.today()
      #If current day of week is before day from df we have to get date from last week
      if today.weekday()<weekdays[day]:
  
        start_week=today - timedelta(days=today.weekday())
        tmp_weekday=start_week+timedelta(days=weekdays[day])
        date_weekday=tmp_weekday-timedelta(days=7)
      else:
        start_week=today - timedelta(days=today.weekday())
        date_weekday= start_week + timedelta(days=weekdays[day])

      return date_weekday
#For other dates in wrong format convert them to date in correct format
def parse_string(date_ago):
  re_date=re.search('^[0-9]{1,2}-[0-9]{1,2}-[0-9]{4}$',date_ago)
  date_proper=None
  
  if re_date==None:
    
    if date_ago=='yesterday':
      date_proper=datetime.now()-timedelta(days=1)
      
    else:
      date_str=re.sub('[0-9]','',date_ago).strip()
      
      date_num=re.findall('\d+',date_ago)
      if len(date_num)!=0:
        date_num=re.findall('\d+',date_ago)[0]


      if date_str=='weeks ago':
        days_w=int(date_num)*7
        date_proper=datetime.now()-timedelta(days=days_w)
      elif date_str=='a week ago':
        days_w=7
        date_proper=datetime.now()-timedelta(days=days_w)
      elif date_str=='a month ago':
        date_proper=date.today() + relativedelta(months=-1)
      elif date_str=='hours ago':
        date_proper=date.today()
      else:
        
        
        
        date_proper=parse_day_to_date(date_str)
    date_proper=date_proper.strftime('%d-%m-%Y')

  else:
    date_proper=date_ago
  return date_proper
  
    


In [27]:
mental_health_df['date_correct']=mental_health_df['date'].apply(lambda x : parse_string(x))

In [28]:
mental_health_df

,category,date,title,text,date_correct
0,Anxiety,22-10-2019,what physical feelings of anxiety do you get?,"hey there, what physical symptoms of anxiety d...",22-10-2019
1,Anxiety,17-11-2015,Forums etiquette: give support to receive support,"Hi everyone, particularly any new members who ...",17-11-2015
2,Anxiety,26-12-2021,Health Anxiety - How Do You Manage?,Hello I am 59 years of age - I have had awful ...,26-12-2021
3,Anxiety,14-10-2019,The Dark Spiral of OCD and Bipolar,"Hi, first time posting on here (or anywhere fo...",14-10-2019
4,Anxiety,19-10-2022,Anxiety about the future,"Hi everyone, I'm 18 years old and recently mov...",19-10-2022
...,...,...,...,...,...
17663,Suicidal thoughts and self-harm,07-09-2015,PLEASE READ THIS FIRST: posting in this section,Life gets pretty hard sometimes and many peopl...,07-09-2015
17664,Suicidal thoughts and self-harm,09-04-2017,13 Reasons Why,I'm not sure if anyone has seen the Netflix se...,09-04-2017
17665,Suicidal thoughts and self-harm,27-11-2016,Feeling really down and struggling,"Hi guys, I am new to Beyond Blue and to comple...",27-11-2016
17666,Suicidal thoughts and self-harm,07-09-2015,PLEASE READ THIS FIRST: posting in this section,Life gets pretty hard sometimes and many peopl...,07-09-2015


In [29]:
mental_health_df=mental_health_df.drop('date_correct', axis=1)

In [30]:
mental_health_df

,category,date,title,text
0,Anxiety,22-10-2019,what physical feelings of anxiety do you get?,"hey there, what physical symptoms of anxiety d..."
1,Anxiety,17-11-2015,Forums etiquette: give support to receive support,"Hi everyone, particularly any new members who ..."
2,Anxiety,26-12-2021,Health Anxiety - How Do You Manage?,Hello I am 59 years of age - I have had awful ...
3,Anxiety,14-10-2019,The Dark Spiral of OCD and Bipolar,"Hi, first time posting on here (or anywhere fo..."
4,Anxiety,19-10-2022,Anxiety about the future,"Hi everyone, I'm 18 years old and recently mov..."
...,...,...,...,...
17663,Suicidal thoughts and self-harm,07-09-2015,PLEASE READ THIS FIRST: posting in this section,Life gets pretty hard sometimes and many peopl...
17664,Suicidal thoughts and self-harm,09-04-2017,13 Reasons Why,I'm not sure if anyone has seen the Netflix se...
17665,Suicidal thoughts and self-harm,27-11-2016,Feeling really down and struggling,"Hi guys, I am new to Beyond Blue and to comple..."
17666,Suicidal thoughts and self-harm,07-09-2015,PLEASE READ THIS FIRST: posting in this section,Life gets pretty hard sometimes and many peopl...
